In [2]:
import requests
import json
import pandas as pd
import numpy as np

In [ ]:
df_games_rawg=pd.DataFrame()
df_games_rawg

In [ ]:

name=['action', 'adventure', 'arcade', 'casual', 'indie', 'platformer', 'ps5', 'puzzle', 'racing', 'shooter', 'simulation','sports', 'strategy', 'xb1']
for i in name:
  df = pd.read_csv(f"rawg_games(2010-2025),{i}.csv")
  df_games_rawg = pd.concat([df_games_rawg, df], ignore_index=True, sort=False)
df_games_rawg


In [ ]:
df_games_rawg.drop_duplicates(subset="gid", keep="first", inplace=True)
df_games_rawg.reset_index(drop=True, inplace=True)
df_games_rawg

In [ ]:
df_games_rawg['developers'] = ""
df_games_rawg['publishers'] = ""

In [ ]:
API_KEY = "d081a22bc7714c7f9d5d381403505669"
def search_dev_pub(gid, api_key):
    url = f"https://api.rawg.io/api/games/{gid}"
    r = requests.get(url, params={"key": api_key}, timeout=10)
    if r.status_code != 200:
        return {"developers": "", "publishers": ""}
    data = r.json()
    dev_names = ", ".join([d.get("name", "") for d in (data.get("developers") or [])])
    pub_names = ", ".join([p.get("name", "") for p in (data.get("publishers") or [])])
    return {"developers": dev_names, "publishers": pub_names}
n = df_games_rawg.shape[0]
for i in range(n):
    gid = df_games_rawg.iloc[i, 0]
    if pd.isna(gid):
        continue
    res = search_dev_pub(int(gid), API_KEY)
    df_games_rawg.at[i, 'developers'] = res['developers']
    df_games_rawg.at[i, 'publishers'] = res['publishers']
OUT_CSV = "rawg_with_devs_pubs.csv"
df_games_rawg.to_csv(OUT_CSV, index=False, encoding="utf-8")

In [ ]:
df_games_rawg= pd.read_csv(f"rawg_with_devs_pubs.csv")
df_games_rawg.head()

In [ ]:
df_games_rawg['playtime'] = ""
df_games_rawg['reviews_count'] = ""

In [ ]:
API_KEY = "d081a22bc7714c7f9d5d381403505669"
def search_play_reviews(gid, api_key):
    url = f"https://api.rawg.io/api/games/{gid}"
    r = requests.get(url, params={"key": api_key}, timeout=10)
    if r.status_code != 200:
        return {"playtime": "", "reviews_count": ""}
    data = r.json()
    playtime = data.get("playtime", "")
    reviews = data.get("reviews_count") or data.get("ratings_count") or ""
    return {"playtime": playtime, "reviews_count": reviews}
n = df_games_rawg.shape[0]
for i in range(n):
    gid = df_games_rawg.iloc[i, 0]
    if pd.isna(gid):
        continue
    res = search_play_reviews(int(gid), API_KEY)
    df_games_rawg.at[i, 'playtime'] = res['playtime']
    df_games_rawg.at[i, 'reviews_count'] = res['reviews_count']

OUT_CSV = "rawg_with_playtime_reviews.csv"
df_games_rawg.to_csv(OUT_CSV, index=False, encoding="utf-8")

In [ ]:
df_games_rawg.drop(columns=['gid'], inplace=True)

In [ ]:
df_games_rawg['source'] = 'rawg.com'
df_games_rawg

In [ ]:
col = df_games_rawg.pop('source')
df_games_rawg.insert(0, 'source', col)
df_games_rawg

In [ ]:
df_games_rawg.to_csv("rawg_games_after_cleaning_developers_publisher_playtime_reviews_count.csv", index=False, encoding="utf-8")